Notebook which will:
- Obtain alert photometry for named SN and create alert.
- Check whether this passes through a filter.

In [1]:
import requests, os
from ampel.log.AmpelLogger import AmpelLogger
from ampel.contrib.hu.t0.SimpleDecentFilter import SimpleDecentFilter
from ampel.contrib.hu.t0.RedshiftCatalogFilter import RedshiftCatalogFilter

from ampel.ztf.alert.ZiAlertSupplier import ZiAlertSupplier

import numpy as np

In [2]:
logger = AmpelLogger.get_logger()

In [3]:
# Access to the AMPEL data archive assumes an individual *archive token* which can be obtained from 
# https://ampel.zeuthen.desy.de/live/dashboard/tokens
# This currently builds on GitHub authorization for AmpelProject members.
token = os.environ["ARCHIVE_TOKEN"]

In [4]:
filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-healpix/broken_ampel_200115.csv"

# Name of ZTF objects to lookup
snnames = np.genfromtxt(filename, delimiter=",", dtype=None,encoding="UTF-8")

print(snnames)

['ZTF18acckpja' 'ZTF19aaevvnk' 'ZTF20aafqpdz' 'ZTF20aafmdyo'
 'ZTF20aafmdyo' 'ZTF20aafqtmj' 'ZTF20aafqpyx' 'ZTF20aafduuu'
 'ZTF20aafqtun' 'ZTF20aafqqqt' 'ZTF19aaevnir' 'ZTF20aafqpum'
 'ZTF20aafqvyc' 'ZTF20aafrftb' 'ZTF20aafqxxd' 'ZTF20aafrelr'
 'ZTF19aagymlm' 'ZTF20aaeodhh' 'ZTF20aafvbnj' 'ZTF19aamftdx'
 'ZTF20aafvasg' 'ZTF20aaetqwv' 'ZTF20aafvash' 'ZTF20aafukwm']


In [36]:
def api_to_alert(name, token, tags=['FilterTest']):
    """
    Retrieve the alert history of a SN and return all recreated alerts.    
    """
    
    # Setup connection
    endpoint = "https://ampel.zeuthen.desy.de/api/ztf/archive/v3/object/{}/alerts?with_history=false".format(name)

    header = {"Authorization": "bearer "+token}

    response = requests.get(endpoint, headers=header )
    
    if not response.ok:
        print('... failed to get alert')
        return None
    
    return [ZiAlertSupplier.shape_alert_dict( a, tags ) for a in response.json()]

In [37]:
alert_array = []
for snname in snnames:
    alerts = api_to_alert(snname, token)
    print(f'{snname}: Found {len(alerts)} alerts.')
    alert_array.append(alerts)

ZTF18acckpja: Found 59 alerts.
ZTF19aaevvnk: Found 8 alerts.
ZTF20aafqpdz: Found 1 alerts.
ZTF20aafmdyo: Found 26 alerts.
ZTF20aafmdyo: Found 26 alerts.
ZTF20aafqtmj: Found 4 alerts.
ZTF20aafqpyx: Found 3 alerts.
ZTF20aafduuu: Found 9 alerts.
ZTF20aafqtun: Found 7 alerts.
ZTF20aafqqqt: Found 8 alerts.
ZTF19aaevnir: Found 4 alerts.
ZTF20aafqpum: Found 3 alerts.
ZTF20aafqvyc: Found 2 alerts.
ZTF20aafrftb: Found 1 alerts.
ZTF20aafqxxd: Found 3 alerts.
ZTF20aafrelr: Found 1 alerts.
ZTF19aagymlm: Found 4 alerts.
ZTF20aaeodhh: Found 2 alerts.
ZTF20aafvbnj: Found 1 alerts.
ZTF19aamftdx: Found 28 alerts.
ZTF20aafvasg: Found 1 alerts.
ZTF20aaetqwv: Found 17 alerts.
ZTF20aafvash: Found 1 alerts.
ZTF20aafukwm: Found 13 alerts.


Defining parameters for filter.

In [48]:
filter_config = {
    #"gaia_excessnoise_sig_max": 999,
    #"gaia_plx_signif": 3,
    #"gaia_pm_signif": 3,
    #"gaia_rs": 10,
    #"gaia_veto_gmag_max": 20,
    #"gaia_veto_gmag_min": 9,
    "min_ndet": 1, # should be 2
    "min_tspan": -1,
    "max_tspan": 365,
    #min_archive_tspan: -99
    "max_archive_tspan": 0.1,
    "min_drb": 0.3,
    "min_gal_lat": 0,
    "min_rb": 0.0,
    "min_sso_dist": 20,
    "ps1_confusion_rad": 3,
    "ps1_confusion_sg_tol": 0.1,
    "ps1_sgveto_rad": 1,
    "ps1_sgveto_th": 0.8,
    "max_fwhm": 5.5,
    # max_elong: 2
    "max_elong": 1.4,
    "max_magdiff": 1,
    "max_nbad": 2
}

In [49]:
t0filter = SimpleDecentFilter( **filter_config, logger=logger )
t0filter.post_init()


2023-05-25 14:09:14 SimpleDecentFilter:59 INFO
 Using min_ndet=1
 Using min_tspan=-1.0
 Using max_tspan=365.0
 Using min_archive_tspan=0.0
 Using max_archive_tspan=0.1
 Using min_drb=0.3
 Using min_rb=0.0
 Using max_fwhm=5.5
 Using max_elong=1.4
 Using max_magdiff=1.0
 Using max_nbad=2
 Using min_sso_dist=20.0
 Using min_gal_lat=0.0
 Using ps1_sgveto_rad=1.0
 Using ps1_sgveto_th=0.8
 Using ps1_confusion_rad=3.0
 Using ps1_confusion_sg_tol=0.1


In [50]:

trigger_jd = 2458863.682951

In [51]:
def process(alerts):

    pass_array = []
    rejection_reason_array = []

    for alert in alerts:
        passes = True
        alert_jds = np.array(alert.get_values("jd", filters=t0filter.filter_pps))

        predetection_jds = [jd for jd in alert_jds if jd < trigger_jd]

        if len(predetection_jds) > 0:
            passes = False
            pass_array.append(passes)
            rejection_reason_array.append("predetection")
            continue



        foo = t0filter.process(alert)
        if not foo:
            #print(foo)
            passes = False
            pass_array.append(passes)
            rejection_reason_array.append("simpledecent")
            continue

        pass_array.append(passes)
        #print(foo)

    print("total:", np.any(pass_array), pass_array)
    #print("\t\t\t", rejection_reason_array)

In [52]:
for i, snname in enumerate(snnames):
    print(snname, end =" ")
    process(alert_array[i])

ZTF18acckpja total: False [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
ZTF19aaevvnk total: False [False, False, False, False, False, False, False, False]
ZTF20aafqpdz total: False [False]
ZTF20aafmdyo total: False [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
ZTF20aafmdyo total: False [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
ZTF20aafqtmj total: False


2023-05-25 14:09:16 SimpleDecentFilter:225 INFO [archive_tspan=486.8113425998017]

2023-05-25 14:09:16 SimpleDecentFilter:225 INFO [archive_tspan=494.6971875000745]

2023-05-25 14:09:16 SimpleDecentFilter:225 INFO [archive_tspan=678.026863399893]

2023-05-25 14:09:16 SimpleDecentFilter:225 INFO [archive_tspan=700.0343287000433]

2023-05-25 14:09:16 SimpleDecentFilter:213 INFO [elong=1.4753451347351074]

2023-05-25 14:09:16 SimpleDecentFilter:225 INFO [archive_tspan=730.942951399833]

2023-05-25 14:09:16 SimpleDecentFilter:225 INFO [archive_tspan=736.9613888999447]

2023-05-25 14:09:16 SimpleDecentFilter:225 INFO [archive_tspan=738.895011600107]

2023-05-25 14:09:16 SimpleDecentFilter:225 INFO [archive_tspan=739.9269213001244]

2023-05-25 14:09:16 SimpleDecentFilter:225 INFO [archive_tspan=742.0206249998882]

2023-05-25 14:09:16 SimpleDecentFilter:225 INFO [archive_tspan=743.885486099869]

2023-05-25 14:09:16 SimpleDecentFilter:225 INFO [archive_tspan=757.9257291997783]

2023-05-25 14:

In [12]:
for a in alerts:
    latest = a.datapoints[0]
    # drb
    try:
        print('drb', latest['drb'])
    except KeyError:
        print('no drb')
    # rb
    try:
        print('rb', latest['rb'])
    except KeyError:
        print('no rb')
    # isdiffpos
    try:
        print('isdiffpos', latest['isdiffpos'])
    except KeyError:
        print('no isdiffpos')


drb 0.9851456880569458
rb 0.618571400642395
isdiffpos t
drb 0.9999991655349731
rb 0.8242856860160828
isdiffpos t
drb 0.9996000528335571
rb 0.46714285016059875
isdiffpos t
drb 0.9997393488883972
rb 0.5571428537368774
isdiffpos t
drb 0.9999111890792847
rb 0.8428571224212646
isdiffpos t
drb 0.9999647736549377
rb 0.8357142806053162
isdiffpos t
drb 0.9970144033432007
rb 0.7614285945892334
isdiffpos t
drb 0.9983726143836975
rb 0.6842857003211975
isdiffpos t
drb 0.9981058835983276
rb 0.7914285659790039
isdiffpos t
drb 0.9999994039535522
rb 0.9314285516738892
isdiffpos t
drb 0.9996236562728882
rb 0.9128571152687073
isdiffpos t
drb 0.9999997615814209
rb 0.9385714530944824
isdiffpos t
drb 0.9999539256095886
rb 0.9085714221000671
isdiffpos t


In [53]:
alerts[0].datapoints

({'jd': 2458864.0659259,
  'fid': 2,
  'pid': 1109565924415,
  'diffmaglim': 19.72231674194336,
  'programid': 2,
  'candid': 1109565924415015004,
  'isdiffpos': 't',
  'tblid': 4,
  'nid': 1109,
  'rcid': 44,
  'field': 380,
  'xpos': 963.0435791015625,
  'ypos': 2300.962158203125,
  'ra': 250.1394082,
  'dec': -8.6740589,
  'magpsf': 19.557470321655273,
  'sigmapsf': 0.1663540005683899,
  'chipsf': 1.578667163848877,
  'magap': 19.33880043029785,
  'sigmagap': 0.2721000015735626,
  'distnr': 2.3860857486724854,
  'magnr': 22.381999969482422,
  'sigmagnr': 0.2280000001192093,
  'chinr': 0.7580000162124634,
  'sharpnr': -0.2029999941587448,
  'sky': 0.020566310733556747,
  'magdiff': -0.21866999566555023,
  'fwhm': 3.2799999713897705,
  'classtar': 0.9789999723434448,
  'mindtoedge': 779.5377807617188,
  'magfromlim': 0.383515864610672,
  'seeratio': 0.7646490931510925,
  'aimage': 0.8330000042915344,
  'bimage': 0.7390000224113464,
  'aimagerat': 0.2539634108543396,
  'bimagerat': 0.2

### Explore the RedshiftFilter

In [14]:
zfilter_config = {
    "catalog_match_radius": 30,
    "min_z": 0.002,
    "max_z": 0.025,
    "max_tspan": 5.,        # Max total detection age in alert
    "max_archive_tspan": 5.,        # Max total detection age in IPAC DB
    "min_rb": 0.3,             # real bogus score
    "min_drb": 0.995,         # deep learning real bogus score 
    # Should be set during proper install
    "resource": {"ampel-ztf/catalogmatch":"https://ampel.zeuthen.desy.de/api/catalogmatch/"},
}

In [15]:
zfilter = RedshiftCatalogFilter( **zfilter_config, logger=logger )
zfilter.post_init()

In [16]:
for alert in alerts:
    foo = zfilter.process(alert)
    print(alert.datapoints[0]['jd'], foo)

2458864.0659259 None
2459335.9309375 None
2459337.9111574 None
2459337.9413194 None
2459339.8768866 None
2459342.8762731 None
2459342.9180556 None
2459345.8988542 None
2459345.9414352 None
2460087.7539699 None
2460087.8678125 None
2460089.8064815 None
2460089.8441782 None


In [17]:
alerts[0].datapoints[0]['jd']

2458864.0659259